In [3]:
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")

from MARScore.utils import * 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import hdbscan
from sklearn.metrics import make_scorer
from random import seed
from datasets_loaders.loaders import load_billsum

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

In [4]:
billsum = load_billsum()
subset = billsum.iloc[:50, :]

In [10]:
def DBCV(model, X, y=None):
    preds = model.fit_predict(X)
    return hdbscan.validity.validity_index(X, preds) if len(set(preds)) > 1 else float('nan')

def HDBScanFinetune(vectors, 
                    min_samples=[10,30,50,60,100], 
                    min_cluster_size=[100,200,300,400,500,600],
                    cluster_selection_method=['eom','leaf'],
                    seed_num=0, 
                    verbose=True):
    
    #model setup
    hdb = hdbscan.HDBSCAN(gen_min_span_tree=True)

    # specify parameters and distributions to sample from
    param_dist = {'min_samples': min_samples,
                  'min_cluster_size':min_cluster_size,  
                  'cluster_selection_method':cluster_selection_method
                 }

    #validity_scroer = "hdbscan__hdbscan___HDBSCAN__validity_index"
    validity_scorer = make_scorer(DBCV, greater_is_better=True)

    #parameters research
    """
    random_search = GridSearchCV(hdb,
                                 param_grid=param_dist,
                                 scoring=validity_scorer)
    """
    
    n_iter_search = 2
    random_search = RandomizedSearchCV(hdb,
                                       param_distributions=param_dist,
                                       n_iter=n_iter_search,
                                       scoring=validity_scorer,
                                       random_state=seed(seed_num))
    
    random_search.fit(vectors)

    if verbose:
        print(f"Best Parameters {random_search.best_params_}")
        print(f"DBCV score :{random_search.best_estimator_.relative_validity_}")
    return {"best_params": random_search.best_params_, "dbcv_score":random_search.best_estimator_.relative_validity_}

In [ ]:
def to_shape(a, shape):
    x_, y_, z_ = shape
    x, y, z = len(a), len(a[0]), len(a[0][0])
    x_pad = (x_-x)
    y_pad = (y_-y)
    z_pad = (z_-z)
    return np.pad(a,((0, x_pad),
                     (0, y_pad),
                     (0, z_pad)),
                  mode = 'constant')

In [49]:
def to_shape(a, shape):
    x_, y_ = shape
    x, y = len(a), len(a[0])
    x_pad = (x_-x)
    y_pad = (y_-y)
    return np.pad(a,((0, x_pad),
                     (0, y_pad)),
                  mode = 'constant')

In [6]:
#creation of embeddings
all_v = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    all_v.append(v)

In [60]:
max_dim_1 = np.max([len(x) for x in all_v])
all_v2 = np.array([to_shape(cur_v, (max_dim_1, len(all_v[0][0]))) for cur_v in all_v])
s = all_v2.shape
all_v2 = all_v2.reshape((s[0], s[1]*s[2]))

In [58]:
hdbscan.HDBSCAN().fit(all_v2[0])

HDBSCAN()

In [62]:
HDBScanFinetune(all_v2,
                min_samples=[10,30],
                min_cluster_size=[15,20],
                cluster_selection_method=['eom','leaf'])

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(


KeyboardInterrupt: 